# Simon #metoo step 1

In [ ]:
from IPython.display import HTML
import os

Data was collected using [this method](http://www.simonlindgren.com/stuff/2017/12/13/back-to-future-btf-tweet-collector). It uses the Twitter API to go some days back in time, and then continuously streaming forward in real-time.

The metoo data was collected around the time when the campaign took off, but today we wouldn't be able to get data further back than about a week from now.

The video below shows the setup process for the collector.

In [ ]:
HTML("""
<video width="320" height="240" controls>
  <source src="btf.m4v" type="video/mp4">
</video>
""")

I had collected around 3 million metoo tweets, and they were in `sqlite3` database format. We won't run the parsing process live here but the code below shows how it is done.

In [ ]:
import sqlite3
import pandas as pd
pd.set_option('display.max_colwidth', -1)

## Read sqlite query results into a pandas DataFrame
# conn = sqlite3.connect("tweets.db")
# tweets_df = pd.read_sql_query("SELECT * from tweets", conn)

## Do some cleanupb
#tweets_df = tweets_df.replace({'\n': ' '}, regex=True) # remove linebreaks in the dataframe
#tweets_df = tweets_df.replace({'\t': ' '}, regex=True) # remove tabs in the dataframe
#tweets_df = tweets_df.replace({'\r': ' '}, regex=True) # remove carriage return in the dataframe

For the continued analysis, I start from a csv-file that was created in this way.

In [ ]:
# Read the csv into pandas
metoo_df = pd.DataFrame.from_csv("backup_metoo_171221.csv")

In [ ]:
len(metoo_df)

In [ ]:
metoo_df.head()

Now, let's clean the tweets, and add the clean version of the tweets as a column.

In [ ]:
newlist = []

for tweet in metoo_df['text']:
    #print(tweet)
    tweet = [word.lower() for word in tweet.split() if not word.startswith('#')]
    tweet = [word for word in tweet if not word.startswith('htt')]
    tweet = [word for word in tweet if not word.startswith('@')]
    tweet = [word for word in tweet if not '/' in word]
    tweet = [word for word in tweet if word.isalpha()]
    tweet = [word for word in tweet if not word == "rt"]
    tweet = [word.strip(".") for word in tweet]
    tweet = [word.strip() for word in tweet]
    tweet = [word for word in tweet if not word == "…"]
    tweet = " ".join(tweet)
    newlist.append(tweet)

In [ ]:
tweets = pd.Series(newlist)
tweets

In [ ]:
metoo_df = metoo_df.reset_index(drop=True)

metoo_df['tweets'] = tweets

metoo_df.head()

We want to group the tweets by day, so add a column that says which day they were posted.

In [ ]:
daylist = [day[:10] for day in metoo_df.created_at]
daylist

In [ ]:
days = pd.Series(daylist)
metoo_df['day'] = days

In [ ]:
metoo_df.head()

Group the tweets (tweets) by day.

In [ ]:
tweets_by_day = metoo_df['tweets'].groupby(metoo_df['day'])

View the grouping.

In [ ]:
groups = list(tweets_by_day)

Now `groups` is a list of tuples where element `[0]` is the group name, and element `[1]` is a pandas series of all `text` belonging to the group.

In [ ]:
groups[0][0]

In [ ]:
groups[0][1]

The series is iterable.

In [ ]:
for i in groups[41][1]:
    print(i)

For topic modelling, we create one text document per group (day in this case).

In [ ]:
os.getcwd()

In [ ]:
try:
    os.mkdir("tmfiles")
except:
    print("Directory already exists")

In [ ]:
os.chdir("tmfiles") # run once
os.getcwd()

In [ ]:
for c,i in enumerate(groups):
    o = open(str(groups[c][0]) + ".txt", "w")
    for line in i[1]:
        o.write(line + "\n")

Now we have the topicmodelling documents.

Go back to the previous working directory.

In [ ]:
os.chdir('..')

In [ ]:
os.getcwd()